In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import time
import seaborn as sns
import numpy as np

%matplotlib inline
pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv('..//data/cleaned_pets.csv')

C:\Users\Max-X1\AppData\Local\Temp\ipykernel_64156\3667737777.py:1: DtypeWarning: Columns (17,18,22,23,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('..//data/cleaned_pets.csv')


In [3]:
df.shape

(34010, 42)

In [4]:
# Drop row that has null values throughout all columns
df[df['breeds.primary'].isna()]
df = df.drop(33944)
df.shape

(34009, 42)

Calculate adoption time

In [5]:
# Convert to datetime format
df['published_date'] = pd.to_datetime(df['published_date'], format='%m/%d/%y')
df['status_change_date'] = pd.to_datetime(df['status_change_date'], format='%m/%d/%y')

# Calculate adoption time (time available for adoption)
df['adoption_time'] = (df['status_change_date'] - df['published_date']).dt.days

In [6]:
# Create adopted and adoptable dataframes
adopted = df[(df['year_status_change'] == 2023) & (df['status'] == 'adopted')]
adoptable = df[(df['year_status_change'] == 2023) & (df['status'] == 'adoptable')]

# All Pets Analysis

How many pets were published/put up for adoption in 2023? 

In [7]:
# Subset data for 2023 
published = df[df['year_published'] == 2023]

print('Number of pets placed for adoption: ', published['id'].count())

Number of pets placed for adoption:  34009


How many pets were adopted in 2023?

In [8]:
# Subset data by year 2023 and status = adopted 
adopted = df[(df['year_status_change'] == 2023) & (df['status'] == 'adopted')]

print('Number of pets adopted in 2023: ', adopted['id'].count())

Number of pets adopted in 2023:  29735


How many pets were published but NOT adopted in 2023? 

In [9]:
# Subset year status was changed to not equal to 2023
available = df[df['year_status_change'] != 2023]

print('Pets published but not adopted in 2023: ', available['id'].count())

Pets published but not adopted in 2023:  1251


In [10]:
# Check 
#available.groupby('year_status_change')['id'].count()

How many pets of each species were placed for adoption in 2023?

In [11]:
# Group ids by species and count
published_species = df.groupby('species')['id'].count().reset_index(name='count published').sort_values('count published', ascending=False)

published_species

,species,count published
1,Dog,19677
0,Cat,14116
11,Rabbit,108
3,Guinea Pig,57
10,Pot Bellied,18
6,Horse,12
9,Pig,5
2,Goat,4
7,Miniature Horse,3
4,Hamster,2


How many pets of each species were adopted in 2023?

In [12]:
# Group adopted by species and count ids
adopted_species = adopted.groupby('species')['id'].count().reset_index(name='count adopted').sort_values('count adopted', ascending=False)

adopted_species

,species,count adopted
1,Dog,16982
0,Cat,12592
10,Rabbit,75
3,Guinea Pig,47
9,Pot Bellied,17
5,Horse,7
8,Pig,5
2,Goat,4
6,Miniature Horse,2
11,Rat,2


What is the overall adoption rate for 2023?

In [13]:
# Calculate adoption rate by dividing number of adopted pets by total pets and multipy * 100 to get overall adoption rate
adoption_rate = (adopted['id'].count() / df['id'].count()) * 100

print('Overall Adoption Rate: ', round(adoption_rate, 2),'%')

Overall Adoption Rate:  87.43 %


What is the adoption rate by species? 

In [14]:
# Outer join published species and adopted species
adoption_rate = pd.merge(published_species, adopted_species, on='species', how='outer')

# Fill NaN values with zero
adoption_rate = adoption_rate.fillna(0)

# Convert 'count adopted' column to integer
adoption_rate['count adopted'] = adoption_rate['count adopted'].astype(int)

# Calculate the adoption rate %
adoption_rate['adoption rate %'] = round((adoption_rate['count adopted'] / adoption_rate['count published']) * 100, 2)
adoption_rate.sort_values('adoption rate %', ascending=False)

,species,count published,count adopted,adoption rate %
6,Pig,5,5,100.00
7,Goat,4,4,100.00
11,Rat,2,2,100.00
12,Mule,1,1,100.00
4,Pot Bellied,18,17,94.44
1,Cat,14116,12592,89.20
0,Dog,19677,16982,86.30
3,Guinea Pig,57,47,82.46
2,Rabbit,108,75,69.44
8,Miniature Horse,3,2,66.67


What is the adoption rate by age? What age group has the highest and lowest adoption rate? 

Answer: Highest - Baby, Lowest - Seniors 

In [15]:
# Group id by age and count for published and adopted dfs
pub_age_count = df.groupby('age')['id'].count().reset_index(name='count published')
adopted_age_count = adopted.groupby('age')['id'].count().reset_index(name='count adopted')

# Merge two dfs together
age_adoption_rate = pd.merge(pub_age_count, adopted_age_count, on='age')

# Calculate adoption rate
age_adoption_rate['adoption rate %'] = round((adoption_rate['count adopted'] / adoption_rate['count published']) * 100, 2)

age_adoption_rate.sort_values('adoption rate %', ascending=False)

,age,count published,count adopted,adoption rate %
1,Baby,8565,7668,89.20
0,Adult,16477,14741,86.30
3,Young,7488,6013,82.46
2,Senior,1479,1313,69.44


In [16]:
df.columns

Index(['id', 'organization_id', 'url', 'type', 'species', 'age', 'gender',
       'size', 'coat', 'tags', 'name', 'description', 'status',
       'status_changed_at', 'published_at', 'breeds.primary',
       'breeds.secondary', 'breeds.mixed', 'breeds.unknown', 'colors.primary',
       'colors.secondary', 'colors.tertiary', 'attributes.spayed_neutered',
       'attributes.house_trained', 'attributes.declawed',
       'attributes.special_needs', 'attributes.shots_current',
       'environment.children', 'environment.dogs', 'environment.cats',
       'contact.address.city', 'contact.address.state',
       'contact.address.postcode', 'published_date', 'status_change_date',
       'adoption_time', 'month_published', 'day_published', 'year_published',
       'month_status_change', 'day_status_change', 'year_status_change'],
      dtype='object')

What is the adoption rate by month?

In [17]:
# Create column for adopted months

# Create a new column named 'adopted_month' and initialize all values to 0
df['adopted_month'] = 0  

# Loop through each row in the DataFrame and check for the following conditions
# If true - update adopted_month column with month status was changed, if false - insert zero
for index, row in df.iterrows():
    if row['status'] == 'adopted' and row['year_status_change'] == 2023:
        df.at[index, 'adopted_month'] = row['month_status_change']
    else:
        df.at[index, 'adopted_month'] = 0

What is the average adoption time (in days) per species? Which species has the longest adoption time?

In [18]:
# Calculate average adoption time by species
avg_time = adopted.groupby('species')['adoption_time'].mean().reset_index(name='avg_adoption_time').sort_values('avg_adoption_time', ascending=False)

print('Answer: Rabbits:', round(avg_time['avg_adoption_time'].max()), 'days')
avg_time

Answer: Rabbits: 74 days


,species,avg_adoption_time
10,Rabbit,73.533333
11,Rat,73.500000
5,Horse,50.571429
7,Mule,43.000000
6,Miniature Horse,42.000000
3,Guinea Pig,38.829787
1,Dog,17.992286
9,Pot Bellied,16.235294
0,Cat,15.105702
2,Goat,7.000000


What cities in TN are pets available in TN?

In [29]:
df.columns

Index(['id', 'organization_id', 'url', 'type', 'species', 'age', 'gender',
       'size', 'coat', 'tags', 'name', 'description', 'status',
       'status_changed_at', 'published_at', 'breeds.primary',
       'breeds.secondary', 'breeds.mixed', 'breeds.unknown', 'colors.primary',
       'colors.secondary', 'colors.tertiary', 'attributes.spayed_neutered',
       'attributes.house_trained', 'attributes.declawed',
       'attributes.special_needs', 'attributes.shots_current',
       'environment.children', 'environment.dogs', 'environment.cats',
       'contact.address.city', 'contact.address.state',
       'contact.address.postcode', 'published_date', 'status_change_date',
       'adoption_time', 'month_published', 'day_published', 'year_published',
       'month_status_change', 'day_status_change', 'year_status_change',
       'adopted_month'],
      dtype='object')

# Dog Analysis

In [19]:
dogs = df[df['species'] == 'Dog']
print(dogs.shape)
dogs.groupby('type')['id'].count()

(19677, 43)


type
Dog    19677
Name: id, dtype: int64

How many full breeds vs mixed breeds were listed for adoption?

In [33]:
full = dogs[dogs['breeds.mixed'] == False]
mixed = dogs[dogs['breeds.mixed'] == True]

print('Full Breed Dogs: ', full['id'].count())
print('Mixed Breed Dogs: ', mixed['id'].count())

Full Breed Dogs:  5833
Mixed Breed Dogs:  13844


What are the top 20 breeds listed for adoption? Are they full or mixed breeds?

In [39]:
top_20 = dogs.groupby(['breeds.primary', 'breeds.mixed'])['id'].count().reset_index(name='count').sort_values('count', ascending=False).head(20)
top_20

,breeds.primary,breeds.mixed,count
202,Labrador Retriever,True,1299
225,Mixed Breed,False,809
328,Terrier,True,778
182,Hound,True,767
98,Chihuahua,True,624
159,German Shepherd Dog,True,623
254,Pit Bull Terrier,True,619
36,Beagle,True,581
302,Shepherd,True,527
29,Australian Shepherd,True,465
